#### Introduction to relations and relational algebra
CS 236 <br>
Fall 2024

Michael A. Goodrich and Eric G. Mercer<br>
Brigham Young University <br>
March 2023, Updated Oct 2024
***

Consider the following example from the class slides:
    _Relational Algebra Part 1_

* $R\subset A\times B$
* $R=\{(a,1),(b,2),(c,3),(d,4)\}$

The relation $R$ is represented as a set of mathematical tuples, which uses the textbook's notation. 

In relational databases and in class, the same relation is represented as a table:

| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 

The first row of the table is the __header__. You can think of the elements of the header as assigning names to each column. For example, the first column contains elements from the set $A$. The header contains information about the sets from which the cartesian product is formed, $R\subset A\times B$.

Below the header are rows containing __tuples__. The tuples are the elements of the __set__ the defines the relation, $R=\{(a,1),(b,2),(c,3),(d,4)\}$.

Just like the order of elements in a set doesn't matter

* $\{(a,1),(b,2),(c,3),(d,4)\} = \{(d,4),(b,2),(a,1),(c,3)\}$

The order of the rows in the table doesn't matter (except for the header). The following table represents the same relation as the table above.

| char | int | 
| :-: | :-: | 
| $d$ | $4$ | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 


***

Let's construct a class that represents a relation.  The variables of this function need to match the parts of the table. Thus, there needs to be a variable representing the header and another variable representing the set of tuples. Similar to how the Project 3 started code does it, we'll define the following
- The relation name as a `str`. You won't see the relation name in the starter code for Project 3, but it's useful here since it allows us to track a few things in the tutorial. 
- The relation header will be defined as a list of strings: `list[str]`
- Each tuple in the relation will be defined as a new type called `RelationTuple`
  - The `RelationTuple` will be built using Python's default `tuple` type.  
  - We'll allow both integers and strings to be the elements of a tuple, so look for the vertical bar that says the tuple types can be either an int or a str yielding `RelationTuple = tuple[str | int]`
  - We won't specify how many elements will be in the `RelationTuple` so we'll use Python's `...` to indicate that we can have several tuples yielding `RelationTuple = tuple[str | int, ...]`
- A `__str__` method that returns a string in the correct format.

In [134]:
from tabulate import tabulate # requires the tabulate module
        ### I like using the tabulate environment, but I had to install a
        ### package to make this work. Within the vscode terminal, the 
        ### command to install is "pip3 install tabulate"

RelationTuple = tuple[str | int, ...]
        
class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name to help show how the math works
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value
        
    
R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
print(R)

Q: Relation = Relation('Q',('C','D'),{})
print(Q)

# The following code will cause the __str__ method to crash
# The problem is that tabulate requires strings and the types in the set are ints
# W: Relation = Relation('W',('A'), {1, 2, 3})
# They must be tuples. Later, the trailing comma will be explained
W: Relation = Relation('W',('A'), {(1,), (2,), (3,)})
print(W)


relation name = R
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ c      │     3 │
├────────┼───────┤
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
╘════════╧═══════╛
relation name = Q
╒═════╤═════╕
│ C   │ D   │
╞═════╪═════╡
╘═════╧═════╛
relation name = W
╒═════╕
│   A │
╞═════╡
│   1 │
├─────┤
│   2 │
├─────┤
│   3 │
╘═════╛


The top row in the tables above consist of the relation header. The remaining rows represent the set of tuples contained in the relation. Each line contains a unique tuple.

The order of the tuples in the set of tuples doesn't matter since sets are not ordered, so the relation R2 defined below is the same as R defined above.

In [135]:
R2: Relation = Relation(relation_name = "R2",relation_header = ('char','int'), set_of_tuples = {('b',2),('a',1),('c',3),('d',4)})
print(R2)


relation name = R2
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ c      │     3 │
├────────┼───────┤
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
╘════════╧═══════╛


Interestingly, my machine puts the tuples in R and the tuples in R2 on the same row.

---


Note that testing whether two relations are equal might not work the way our intuition says it should. Consider whether you think the following statement should be true or false.

In [136]:
print(R==R2)

False


The reason is that R and R2 are two different instances of a class, and even though the variables within the two classes are the same the instances are not considered equal. When we print out the `id` information about each instance, we see that the objects have different addresses. A useful LLM prompt is

    what happens when i try to see if two objects are the same in python? For example, suppose I have an object called R and another object called Q and I write"R==Q"

In [137]:
print(id(R))
print(id(R2))

4383608592
4383614208


But we can define an equality operator for the class within the class definition. See

https://stackoverflow.com/questions/1227121/compare-object-instances-for-equality-by-their-attributes

Let's redefine the class to define the `__eq__` function.

In [138]:
from typing import Any

class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name for fun
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )

    
R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
R2: Relation = Relation(relation_name = "R2",relation_header = ('char','int'), set_of_tuples = {('b',2),('a',1),('c',3),('d',4)})
print(R==R2)

True


Defining the `__eq__` function within the class allows us to use the == operator to check whether the contents of the two relations are the same. Notice a few things about the `__eq__` function:

- Two arguments are passed to the function: `self` and `other`. 
  - `self` refers to the instance of the relation class
  - `other` refers to whatever the relation can be compared to. The code above uses the `Any` type to say that we can compare anything to the relation.
- The first line of the function tests whether the `other` object passed in is a `Relation` type. If it's not, the function returns false.
- The function returns the boolean expression that tests whether both headers match as well as whether the set of tuples match.

***


### Two Tricky Problems ###

There are two tricky problems that can consume a lot of time, so let's address each.

#### Problem 1: Modifying copies of objects ####

Let's start with how our intuition thinks copying should work in Python.

In [139]:
string1: str = "Happy"
string2: str = string1 + " Birthday"

print(string1)
print(string2)
print(string1 == string2)
print(id(string1))
print(id(string2))

Happy
Happy Birthday
False
4391131312
4391383792


When we copy a string to another string, the copy creates a new instance of the string. When we modify the copy, only the copy is changed.  We showed three pieces of evidence to support this:
- we printed both strings and saw that they were different
- we used `==` to show that the strings were different
- we printed the `id` of both strings and showed that they were different


Let's see what happens when we copy a relation. We'll create a copy and print the `id` of both the original and the copy.

In [140]:
R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q: Relation = R

print(id(R))
print(id(Q))

4382795344
4382795344


Notice that the two objects have the same `id`. The problem is that Python defaults to creating a **pointer** to the original object. This means that if we modify the copy we also modify the original. We can see this if I overwrite the header of the copy and then print out the header of the original and the copy. 

In [141]:
print(f'Header of R before modification = {R.header}')
print(f'Header of Q before modification = {Q.header}')

Q.header = ['bart', 'lisa']

print(f'Header of R after modification = {R.header}')
print(f'Header of Q after modification = {Q.header}')


Header of R before modification = ('char', 'int')
Header of Q before modification = ('char', 'int')
Header of R after modification = ['bart', 'lisa']
Header of Q after modification = ['bart', 'lisa']


Notice how both the copy and the original are modified. This is because the copy created a new pointer to the original relation, so when we modified the memory to which the pointer referenced the we also modified the original.

We can fix this by defining our own `__copy__` method. A good LLM prompt to explore more is:

     is there a dunder method in python that describes how to perform a copy operation of an object

If you want to avoid this problem, use `deepcopy`

In [142]:
from copy import deepcopy

R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q: Relation = deepcopy(R)

print(id(R))
print(id(Q))

print(f'Header of R before modification = {R.header}')
print(f'Header of Q before modification = {Q.header}')

Q.header = ['bart', 'lisa']

print(f'Header of R before modification = {R.header}')
print(f'Header of Q before modification = {Q.header}')

Q.header = ['bart', 'lisa']

print(f'Header of R after modification = {R.header}')
print(f'Header of Q after modification = {Q.header}')

4382798032
4382799856
Header of R before modification = ('char', 'int')
Header of Q before modification = ('char', 'int')
Header of R before modification = ('char', 'int')
Header of Q before modification = ['bart', 'lisa']
Header of R after modification = ('char', 'int')
Header of Q after modification = ['bart', 'lisa']


---


#### Side Effects and Problem 1: Some Programming Best Practices

**Note that some of the information shared here was culled from copilot in response to a prompt on differences in initalizing member variables in the starter code.**

Starter code is provided for project 3. It has a subtle but important difference with the definition of the class above. Specifically, assigning the `relation_header` to the class variable `self.header`, which occurs in `__init__`. In the code above, the assignment is

`self.header: list[str] = relation_header`

but in the starter code the assigment is 

`self.header = list(relation_header)`

What's the difference? Let's experiment with some variables outside of the definition of the Relation class. 

In [143]:
relation_header: list[str] = ['A', 'B', 'C']
header_v1: list[str] = relation_header
header_v2 = list(relation_header)

print(f"The original list has id = {id(relation_header)}")
print(f"The assignment operator in version 1 has id = {id(header_v1)}")
print(f"The assignment operator in version 2 has id = {id(header_v2)}")

The original list has id = 4416579520
The assignment operator in version 1 has id = 4416579520
The assignment operator in version 2 has id = 4370466688


Notice that the id's for the original list and the first version of the header have the same id, but the id's of the original list and the second version of the header have different id's. Stated simply,
 - the assignment `header_v1: list[str] = relation_header` copies a *reference* (e.g., the pointer)
 - the assignment `header_v2 = list(relation_header)` creates a brand new list object

You can tell that the second assignment creates a brand new list object because it uses the formatting that we use whenever we create a new object in Python. The `list(stuff)` creates a new instance of a list object, where `stuff` initializes the list object. We can confirm that a new list object is created by asking Python about the list keyword, as follows:

In [144]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self))

The different types of assignments have some important implications. Copilot lists four advantages of using the assignment `header_v1: list[str] = relation_header`

The main difference between the two versions of the `Relation` class is how the `header` attribute is initialized. In the first version, `self.header` is assigned a copy of the `header` list, while in the second version, `self.header` is assigned the `header` list directly. Here are some reasons why you might choose the second approach:

1. **Memory Efficiency**:
   - By directly assigning `header`, you avoid creating a copy of the list, which can save memory, especially if the list is large.

2. **Performance**:
   - Direct assignment is faster than copying the list, which can be beneficial if this operation is performed frequently.

3. **Mutability Considerations**:
   - If you want changes to the original `header` list to be reflected in the `Relation` instance, direct assignment is necessary. However, this also means that external changes to the `header` list will affect the `Relation` instance, which might not always be desirable.

4. **Simplicity**:
   - The code is slightly simpler and more straightforward with direct assignment.

Be cautious with direct assignment if you need to ensure that the `Relation` instance has its own independent copy of the `header` list. In such cases, copying the list (as in the first version) is safer to avoid unintended side effects from external modifications.


Copilot lists the following advantages of the assignment `header_v2 = list(relation_header)`

The first approach, where `header` is assigned a copy of the `relation_header` list, has several advantages:

1. **Data Integrity**:
   - By copying the list, you ensure that the `Relation` instance has its own independent version of the `header`. This prevents unintended side effects from external modifications to the original list.

2. **Encapsulation**:
   - This approach adheres to the principle of encapsulation, where the internal state of the object is protected from outside interference. Changes to the original `header` list won't affect the `Relation` instance, maintaining the integrity of the object's state.

3. **Predictability**:
   - It makes the behavior of the `Relation` class more predictable. Users of the class can be confident that once a `Relation` object is created, its `header` won't change unless explicitly modified through the class's methods.

4. **Debugging**:
   - It can simplify debugging. If the `header` list is modified unexpectedly, you can be sure that the modification happened within the `Relation` class, not from an external source.

5. **Immutability**:
   - If the `header` list is intended to be immutable after the `Relation` object is created, copying it ensures that the original list cannot be changed, reinforcing this immutability.

Overall, the first approach provides a higher level of safety and robustness, especially in scenarios where the `header` list should remain unchanged after the `Relation` object is initialized.

Let's quickly illustrate by changing `relation_header` and printing out all versions.

In [145]:
relation_header: list[str] = ['A', 'B', 'C']
header_v1: list[str] = relation_header
header_v2 = list(relation_header)
print(f"** Before change **")
print(f"relation_header = {relation_header}")
print(f"header_v1 = {header_v1}")
print(f"header_v2 = {header_v2}")

relation_header[0] = 'new'
relation_header[1] = 'newer'
relation_header.append('newest')
print(f"\n** After change **")
print(f"relation_header = {relation_header}")
print(f"header_v1 = {header_v1}")
print(f"header_v2 = {header_v2}")

** Before change **
relation_header = ['A', 'B', 'C']
header_v1 = ['A', 'B', 'C']
header_v2 = ['A', 'B', 'C']

** After change **
relation_header = ['new', 'newer', 'C', 'newest']
header_v1 = ['new', 'newer', 'C', 'newest']
header_v2 = ['A', 'B', 'C']


Stated simply, if someplace else in your code can change a variable that you used to initialize a class variable, create a copy of that variable when you initialize the class instead of creating a reference to it.

---

#### Problem 2: Creating relations with one-ples or empty relations ####

A tuple can have only a single element in it, what Goodrich jokingly called a "one-ple" in class. There is a tricky problem that comes up when we try to create a tuple with only a single element. Consider the following:

In [146]:

R: Relation = Relation('R',('char'),{('a')}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(R)


relation name = R
╒═════╕
│ c   │
╞═════╡
│ a   │
╘═════╛


Notice how the only the "c" from the attribute name "char" is printed out. This can be a difficult bug to deal.

It gets worse if we try to create a relation with no tuples in the set

In [147]:
Q: Relation = Relation('Q',('char'),{}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(Q)

relation name = Q
╒═════╤═════╤═════╤═════╕
│ c   │ h   │ a   │ r   │
╞═════╪═════╪═════╪═════╡
╘═════╧═════╧═════╧═════╛


That's not what we intended at all. We can get around this by adding a comma after the single element of the tuple (after both the 'char' and the 'a')...

In [148]:
P: Relation = Relation('P',('char',),{('a',)}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(P)


relation name = P
╒════════╕
│ char   │
╞════════╡
│ a      │
╘════════╛


... and by adding a single comma after the header when we create a relation with the set of tuples empty

In [149]:
Q: Relation = Relation('Q',('char',),{}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(Q)


relation name = Q
╒════════╕
│ char   │
╞════════╡
╘════════╛


A good LLM prompt to understand this issue better is

     Why do i have to add a comma after a list with only one element in python? 

--- 

### Adding relational algebra operators to the Relation class

We are now in a position to start filling in the rest of the Relation class.

I want to be able to apply the _relational operators_ to any relation or pair of relations. Using good object-oriented programming style, I'll add the relational operators as functions to the class.

Let's begin by creating our own error handler a la the starter code


In [150]:
class IncompatibleOperandError(Exception):
    def __init__(self, msg: str) -> None:
        super().__init__(msg)



Let's begin with the union operator

### Union ###
Consider the relation R defined as before
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 

and consider a new relation Q defined as
| char | int | 
| :-: | :-: | 
| $f$ | $3$ |

The union $P\cup Q$ is possible since the headers match, and is
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 
| $f$ | $3$ |  

Let's check in the code.

In [151]:
class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name for fun
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        name: str = self.name + "\u222A" + other.name
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(name,header,set_of_tuples) # Create a new relation
        


In [152]:
    
R = Relation(relation_name = 'R',relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q = Relation('Q',('char','int'),{('f',3),}) # Notice the comma after the ('f',3) tuple. This create a set of tuples, with only one element in the set

P: Relation = R.union(Q)
print(P) # I can see that the relation name is a union when I print it out

relation name = R∪Q
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
├────────┼───────┤
│ c      │     3 │
├────────┼───────┤
│ f      │     3 │
╘════════╧═══════╛


Let's check the error handling

In [153]:
try:
    P.union('mystring')
except IncompatibleOperandError as e:
    print(e) 

Tried to union a relation with a <class 'str'>


In [154]:
W: Relation = Relation('Q',('bart','lisa'),{('f',3),}) # Notice the comma after the ('f',3) tuple. This create a set of tuples, with only one element in the set
try:
    P.union(W)
except IncompatibleOperandError as e:
    print(e) 

Tried to union two relations with different headers


Notice that when we perform the union we don't add the tuples to R. Instead, we create a new set of tuples and return a brand new relation with those tuples.

***


#### Project ####

We'll only implement a portion of this operator since we want to implement the rest in Project 3. Specifically, this project operation will only work to project to a single column.
 
Consider the relation $P = R\cup Q$ defined as before
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ |
| $f$ | $3$ | 

We want to compute $\pi_{char}(P)$, which does two things. 
* First, it creates a new relation.
* Second, it populates the new relation with the char column.

The result is the relation $\pi_{char}(P)$
| char | 
| :-: |
| $a$ |
| $b$ |
| $c$ |
| $d$ |
| $f$ |

***
Here's the code

In [155]:
class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name for fun
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        name: str = self.name + "\u222A" + other.name
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(name,header,set_of_tuples) # Create a new relation

    def project(self,column_header: str) -> 'Relation':
        # Only a portion of this function is implemented. 
        # Specifically, only the portion that projects onto a single column.
        # You have to implement the rest of this function in the Project 3
        
        # First, check the precondition
        # The precondition for the project operator is that the column attribute
        # must exist in the set of attributes
        if column_header not in set(self.header):
            raise IncompatibleOperandError(f"Tried to project onto {column_header}, which isn't an attribute of the relation")
        
        # Second, create a new relation that is the output of the projection operator.
        # THe relation needs a name, a header, and the set of tuples
        new_name = "\u03C0" + "_{" + column_header + "}(" + self.name + ")" # The \u03c0 is a special code for a union symbol
        new_header = (column_header,) # Notice the comma after "column_header", which forces the header to be a tuple
        header_index = self.header.index(column_header) # Get the index of the header that matches the column you want
        new_tuples: set[RelationTuple] = {(rtuple[header_index],1) for rtuple in self.set_of_tuples}
        new_relation = Relation(new_name,new_header,new_tuples) # Create the relation
        return new_relation 


Let's now look at the output of the project operator. 

In [156]:
R: Relation = Relation(relation_name = 'R',relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q: Relation = Relation('Q',('char','int'),{('f','3'),}) # Observe the comma after the ('f',3) tuple. This forces python to make the tuple the lone element of a set

P: Relation = R.union(Q)
M: Relation = P.project('char')
print(M)


relation name = π_{char}(R∪Q)
╒════╤════════╕
│    │   char │
╞════╪════════╡
│ a  │      1 │
├────┼────────┤
│ d  │      1 │
├────┼────────┤
│ b  │      1 │
├────┼────────┤
│ c  │      1 │
├────┼────────┤
│ f  │      1 │
╘════╧════════╛


Let's project onto the 'int' column instead. Why does projecting onto the 'char' column produce five tuples but projecting onto the 'char' column only produce four tuples?

In [157]:
P = R.union(Q)
M = P.project('int')
print(M)

relation name = π_{int}(R∪Q)
╒════╤═══════╕
│    │   int │
╞════╪═══════╡
│  3 │     1 │
├────┼───────┤
│  2 │     1 │
├────┼───────┤
│  3 │     1 │
├────┼───────┤
│  1 │     1 │
├────┼───────┤
│  4 │     1 │
╘════╧═══════╛


The answer is that the set of tuples is a set, and sets don't have repeats. 

Without ignoring repeats, we have the relation $P$ defined as
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ |
| $f$ | $3$ | 

yielding $\pi_{int}(P)$ 
| int | 
| :-: | 
| $1$ |
| $2$ | 
| $3$ | 
| $4$ |
| $3$ |

but the "3" appears in the set twice, so the correct answer is
$\pi_{int}(P)$ 
| int | 
| :-: | 
| $1$ |
| $2$ | 
| $3$ | 
| $4$ |


---

You can ignore the rest of this notebook. I'm using it to create material for the slides in class.

#### What is a database? ###

Consider the following datalog program

<pre>
Schemes:
   childOf(P, C)       # P = Parent, C = Child 
   marriedTo(S1, S2)   # S1 = Spouse 1, S2 = Spouse 2

Facts:<br/>
   marriedTo('Jack', 'Jill').
   marriedTo('Annie', 'Aaron').
   marriedTo('Molly', 'Matt').
   childOf('Jack', 'Matt').
   childOf('Annie', 'Molly').   
   childOf('Matt', 'Nate').

Rules:
   childOf(Y, X) :- childOf(Z, X), marriedTo(Y, Z).
   marriedTo(X, Y) :- marriedTo(Y, X).

Queries:
   marriedTo('Jack', 'Jill')?
   childOf('Aaron', 'Molly')? 
   childOf(P, 'Nate')?
</pre>


This datalog program should create a database with two relations. Let's create them. I'm going to do this without having the _name_ variable in the `Relation` class since this matches the starter code more closely.

In [158]:
class Relation:
    def __init__(self, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.header = list(relation_header)
        self.set_of_tuples = set(set_of_tuples)

    def __str__(self) -> str:
        value: str = tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(header,set_of_tuples) # Create a new relation

    def project(self,column_header: str) -> 'Relation':
        # Only a portion of this function is implemented. 
        # Specifically, only the portion that projects onto a single column.
        # You have to implement the rest of this function in the Project 3
        
        # First, check the precondition
        # The precondition for the project operator is that the column attribute
        # must exist in the set of attributes
        if column_header not in set(self.header):
            raise IncompatibleOperandError(f"Tried to project onto {column_header}, which isn't an attribute of the relation")
        
        # Second, create a new relation that is the output of the projection operator.
        # THe relation needs a name, a header, and the set of tuples
        new_header = (column_header,) # Notice the comma after "column_header", which forces the header to be a tuple
        header_index = self.header.index(column_header) # Get the index of the header that matches the column you want
        # I modified this next line to produce "one-ples"
        new_tuples: set[RelationTuple] = {(rtuple[header_index],) for rtuple in self.set_of_tuples} 
        new_relation = Relation(new_header,new_tuples) # Create the relation
        return new_relation 

In [159]:
childOf: Relation = Relation(relation_header=['P','C'],
                             set_of_tuples={('Jack', 'Matt'),
                                             ('Annie', 'Molly'),
                                             ('Matt','Nate')})
marriedTo: Relation = Relation(relation_header=['S1','S2'],
                             set_of_tuples={('Jack', 'Jill'),
                                             ('Annie', 'Aaron'),
                                             ('Molly','Matt')})

print(childOf)
print(marriedTo)

╒═══════╤═══════╕
│ P     │ C     │
╞═══════╪═══════╡
│ Matt  │ Nate  │
├───────┼───────┤
│ Jack  │ Matt  │
├───────┼───────┤
│ Annie │ Molly │
╘═══════╧═══════╛
╒═══════╤═══════╕
│ S1    │ S2    │
╞═══════╪═══════╡
│ Molly │ Matt  │
├───────┼───────┤
│ Annie │ Aaron │
├───────┼───────┤
│ Jack  │ Jill  │
╘═══════╧═══════╛


How do I store this in a database? Since we are saying that a database is just a set of relations, let's define a relation object as a dictionary of relations. We'll use the relation's name as the key in the dictionary.

In [160]:
database: dict['str',Relation] = {'childOf': childOf, 'marriedTo': marriedTo}
for relation_name in database.keys():
    print(f"relation {relation_name} is \n{database[relation_name]}")

relation childOf is 
╒═══════╤═══════╕
│ P     │ C     │
╞═══════╪═══════╡
│ Matt  │ Nate  │
├───────┼───────┤
│ Jack  │ Matt  │
├───────┼───────┤
│ Annie │ Molly │
╘═══════╧═══════╛
relation marriedTo is 
╒═══════╤═══════╕
│ S1    │ S2    │
╞═══════╪═══════╡
│ Molly │ Matt  │
├───────┼───────┤
│ Annie │ Aaron │
├───────┼───────┤
│ Jack  │ Jill  │
╘═══════╧═══════╛


Let's create the relation that we know should form when I want to do a project operation. Let's project the _childOf_ relation onto the _P_ column, 

${\rm answer} = \pi_{P}({\rm database['childOf']})$

When we do the math by hand, we know that we should end up with a relation with one column and "one-ples" of the parents.  I'm calling this the _math_answer_ because this is what happens when we do the project operation by hand. We'll create a relation for this answer we obtained by hand.

In [161]:
math_answer: Relation = Relation(relation_header=['P'],
                             set_of_tuples={('Matt',), ('Jack',), ('Annie',)})
print(math_answer)

╒═══════╕
│ P     │
╞═══════╡
│ Jack  │
├───────┤
│ Matt  │
├───────┤
│ Annie │
╘═══════╛


Let's now run the project function from the relation class to test whether the _project_ function we've implemented in code actually works.

In [162]:
code_answer = database['childOf'].project('P')
print(code_answer)

╒═══════╕
│ P     │
╞═══════╡
│ Jack  │
├───────┤
│ Matt  │
├───────┤
│ Annie │
╘═══════╛


And we can put this into an assert statement to test that the code output matches the math answer.

In [163]:
assert code_answer.__str__() == math_answer.__str__(), "Project function not implemented correctly"